In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import time
import numpy as np
import pandas as pd
from scipy.stats import skew, chisquare, kruskal, ks_2samp, chi2_contingency

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import MiniBatchKMeans, AffinityPropagation
from sklearn.mixture import GaussianMixture

from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

test_df = pd.read_csv(r'..//data//test.csv')
train_df = pd.read_csv(r'..//data//train.csv')
# train_extra_df = pd.read_csv(r'..//data//training_extra.csv')
# train_df = pd.concat([train_df, train_extra_df], ignore_index=True)


target = 'price'

def prepare_data(df: pd.DataFrame, is_train: bool = True):
    """
    Prepares the dataset for training or testing by renaming columns, handling missing values,
    converting categorical and numerical features, and creating new features.
    
    Args:
        df (pd.DataFrame): The input dataframe (train or test).
        is_train (bool): Indicates if the dataframe is training data (default is True).
        
    Returns:
        pd.DataFrame: The processed dataframe.
    """
    
    # Define the column names
    columns = [
        'id', 'brand', 'material', 'size', 'compartments', 
        'laptop_compartment', 'is_waterproof', 'style', 'color', 
        'weight_capacity'
    ]
    
    if is_train:
        columns.append('price')
    
    df.columns = columns
    df = df.drop(columns='id')
    
    # Define the mapping for Size conversion
    size_mapping = {"Small": 1, "Medium": 2, "Large": 3}
    df["size_int"] = df["size"].map(size_mapping).fillna(0).astype(int)
    
    # Handle weight capacity
    df['weight_capacity'] = df['weight_capacity'].fillna(0)
    df['weight_capacity_int'] = df['weight_capacity'].astype(int)
    df['weight_capacity_size'] = df['weight_capacity'] * df['size_int']
    
    # Convert categorical columns
    df['compartments'] = df['compartments'].astype('category')
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    df[cat_cols] = df[cat_cols].astype('category')
    
    # Convert boolean columns to integer type
    df['laptop_compartment'] = df['laptop_compartment'].cat.codes.fillna(-1).astype(int)
    df['is_waterproof'] = df['is_waterproof'].cat.codes.fillna(-1).astype(int)
    
    return df

# Apply function to train and test datasets
train_df = prepare_data(train_df, is_train=True)
test_df = prepare_data(test_df, is_train=False)

In [83]:
def cross_validate_features(models, X, y, kf, verbose=True):
    model_scores = {name: [] for name in models.keys()}
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'verbose': -1,
        'force_row_wise': True
    }

    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        if verbose:
            print(f"Starting Fold {fold}...")
        fold_start_time = time.time()

        X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
        
        for name, model_features in models.items():
            model_start_time = time.time()
            
            train_data = lgb.Dataset(X_train[model_features], label=y_train)
            valid_data = lgb.Dataset(X_valid[model_features], label=y_valid, reference=train_data)
            fit_model = lgb.train(params, train_data, num_boost_round=500, valid_sets=[valid_data])
            y_pred = fit_model.predict(X_valid[model_features], num_iteration=fit_model.best_iteration)
            
            rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
            model_scores[name].append(rmse)
            
            if verbose:
                print(f"{name} Model - Fold {fold} - Training & Prediction time: {time.time() - model_start_time:.2f} seconds")

        if verbose:
            print(f"Total time for Fold {fold}: {time.time() - fold_start_time:.2f} seconds")
            print("-" * 50)

    return pd.DataFrame(model_scores)

In [80]:
train_df['weight_capacity_int'] = train_df['weight_capacity'].astype(int)
train_df['weight_capacity_size'] = train_df['weight_capacity'] * train_df['size_int']

train_df['weight_capacity_binned'] = pd.qcut(train_df['weight_capacity'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# Interaction Terms
train_df['weight_capacity_brand'] = train_df['weight_capacity'] * train_df['brand'].astype('category').cat.codes

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, include_bias=False)
train_df[['weight_capacity_poly_2']] = poly.fit_transform(train_df[['weight_capacity']])[:, 1:2]  # squared term
train_df[['weight_capacity_poly_3']] = poly.transform(train_df[['weight_capacity']])[:, 2:]

# Exponential
train_df['weight_capacity_exp'] = np.exp(train_df['weight_capacity'])

# Reciprocal Transformations
train_df['weight_capacity_inv'] = 1 / (train_df['weight_capacity'] + 1e-6)  # Avoid division by zero

# Principal Component Analysis (PCA)
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
train_df['weight_capacity_pca'] = pca.fit_transform(train_df[['weight_capacity']])
# train_df['weight_capacity_pca2'] = pca.fit_transform(train_df[['weight_capacity', 'color', 'compartments', 'brand']])

In [76]:
baseline_features = ['weight_capacity', 'color', 'compartments', 'brand', 'material', 'is_waterproof']
features_to_try = [
    'weight_capacity_int', 'weight_capacity_size', 
    'weight_capacity_binned', 'weight_capacity_brand', 'weight_capacity_poly_2', 
    'weight_capacity_poly_3','weight_capacity_exp', 'weight_capacity_inv', 
    'weight_capacity_pca'] # , 'weight_capacity_density'

models = {
    "baseline": baseline_features,
    "weight_capacity_int": ['weight_capacity_int'] + baseline_features,
    "weight_capacity_size": ['weight_capacity_size'] + baseline_features,
    "weight_capacity_binned": ['weight_capacity_binned'] + baseline_features,
    "weight_capacity_brand": ['weight_capacity_brand'] + baseline_features,
    "weight_capacity_poly_2": ['weight_capacity_poly_2'] + baseline_features,
    "weight_capacity_poly_3": ['weight_capacity_poly_3'] + baseline_features,
    "weight_capacity_exp": ['weight_capacity_exp'] + baseline_features,
    "weight_capacity_inv": ['weight_capacity_inv'] + baseline_features,
    # "weight_capacity_density": ['weight_capacity_density'] + baseline_features,
    "weight_capacity_pca": ['weight_capacity_pca'] + baseline_features,
    
    # Combination models
    "poly_features": ['weight_capacity_poly_2', 'weight_capacity_poly_3'] + baseline_features,
    "transformed_features": ['weight_capacity_exp', 'weight_capacity_inv'] + baseline_features,
    "interaction_features": ['weight_capacity_brand', 'weight_capacity_size'] + baseline_features,
    # "density_pca_features": ['weight_capacity_density', 'weight_capacity_pca', 'weight_capacity'] + baseline_features,
    "all_features": features_to_try + baseline_features  # Full model
}

In [ ]:

X = train_df.drop(columns=[target])
y = train_df[target]
kf = KFold(n_splits=10, shuffle=True, random_state=42)
result_df = cross_validate_features(models, X, y, kf, verbose=False)
summary_df = pd.DataFrame({
    "Mean RMSE": result_df.mean(),
    "Std RMSE": result_df.std()
})
display(summary_df)

In [72]:
cat_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist() + ['laptop_compartment', 'is_waterproof']

# Loop over each categorical column and map each category to the mean target value
for col in cat_cols:
    # Compute the mean target for each category in the column
    category_target_mean = train_df.groupby(col)[target].mean()
    
    # Replace the original categorical column with the target-encoded column
    train_df[f'{col}_mean'] = train_df[col].map(category_target_mean)

cat_mean_cols = [f'{col}_mean' for col in cat_cols]

In [73]:
train_df.select_dtypes(include=['object', 'category']).columns.tolist() + ['laptop_compartment', 'is_waterproof']

['brand',
 'material',
 'size',
 'compartments',
 'style',
 'color',
 'brand_mean',
 'material_mean',
 'size_mean',
 'compartments_mean',
 'style_mean',
 'color_mean',
 'laptop_compartment',
 'is_waterproof']

In [74]:
cat_mean_cols

['brand_mean',
 'material_mean',
 'size_mean',
 'compartments_mean',
 'style_mean',
 'color_mean',
 'laptop_compartment_mean',
 'is_waterproof_mean']

In [ ]:
train_df.columns

Index(['id', 'Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price'],
      dtype='object')

In [81]:
train_df.columns

Index(['brand', 'material', 'size', 'compartments', 'laptop_compartment',
       'is_waterproof', 'style', 'color', 'weight_capacity', 'price',
       'size_int', 'weight_capacity_int', 'weight_capacity_size', 'brand_mean',
       'material_mean', 'size_mean', 'compartments_mean', 'style_mean',
       'color_mean', 'laptop_compartment_mean', 'is_waterproof_mean',
       'weight_capacity_binned', 'weight_capacity_brand',
       'weight_capacity_poly_2', 'weight_capacity_poly_3',
       'weight_capacity_exp', 'weight_capacity_inv', 'weight_capacity_pca'],
      dtype='object')

In [84]:
models = {
    "baseline": baseline_features,
    "cat_mean_cols": cat_mean_cols,
    "cat_mean_cols + ": cat_mean_cols + baseline_features,
    "all_features +":  features_to_try + baseline_features,
    "all_features ++":  cat_mean_cols + features_to_try + baseline_features
}

X = train_df.drop(columns=[target])
y = train_df[target]
kf = KFold(n_splits=10, shuffle=True, random_state=42)
result_df = cross_validate_features(models, X, y, kf, verbose=False)
summary_df = pd.DataFrame({
    "Mean RMSE": result_df.mean(),
    "Std RMSE": result_df.std()
})
display(summary_df)

,Mean RMSE,Std RMSE
baseline,39.037680,0.063194
cat_mean_cols,39.057084,0.067168
cat_mean_cols +,39.033518,0.064217
all_features +,39.040334,0.062872
all_features ++,39.036105,0.065110


In [85]:
train_df

,brand,material,size,compartments,laptop_compartment,is_waterproof,style,color,weight_capacity,price,...,color_mean,laptop_compartment_mean,is_waterproof_mean,weight_capacity_binned,weight_capacity_brand,weight_capacity_poly_2,weight_capacity_poly_3,weight_capacity_exp,weight_capacity_inv,weight_capacity_pca
0,Jansport,Leather,Medium,7.0,1,0,Tote,Black,11.611723,112.15875,...,80.513439,81.463842,81.572050,Low,11.611723,134.832107,1565.633046,1.103843e+05,0.086120,6.409977
1,Jansport,Canvas,Small,10.0,1,1,Messenger,Green,27.078537,68.88056,...,82.381308,81.463842,81.403489,Very High,27.078537,733.247143,19855.259594,5.755181e+11,0.036930,-9.056836
2,Under Armour,Leather,Small,2.0,1,0,Messenger,Red,16.643760,39.17320,...,81.011644,81.463842,81.572050,Medium,66.575040,277.014745,4610.566922,1.691582e+07,0.060083,1.377940
3,Nike,Nylon,Small,8.0,1,0,Messenger,Green,12.937220,80.60793,...,82.381308,81.463842,81.572050,Medium,25.874441,167.371669,2165.324158,4.154927e+05,0.077296,5.084480
4,Adidas,Canvas,Medium,1.0,1,1,Messenger,Green,17.749338,86.02312,...,82.381308,81.463842,81.403489,Medium,0.000000,315.039016,5591.734125,5.110222e+07,0.056340,0.272362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,Adidas,Leather,Small,9.0,0,0,Tote,Blue,12.730812,129.99749,...,82.006994,81.464702,81.572050,Medium,0.000000,162.073567,2063.328072,3.380036e+05,0.078550,5.290888
299996,Jansport,Leather,Large,6.0,0,1,Tote,Blue,26.633182,19.85819,...,82.006994,81.464702,81.403489,Very High,26.633182,709.326396,18891.619165,3.686753e+11,0.037547,-8.611482
299997,Puma,Canvas,Large,9.0,1,1,Backpack,Pink,11.898250,111.41364,...,81.630864,81.463842,81.403489,Low,35.694749,141.568346,1684.415531,1.470091e+05,0.084046,6.123450
299998,Adidas,Nylon,Small,1.0,0,1,Tote,Pink,6.175738,115.89080,...,81.630864,81.464702,81.403489,Low,0.000000,38.139739,235.541029,4.809378e+02,0.161924,11.845962


In [38]:
def cross_validate_lightgbm_feature_importances(
    params, X, y, kf, num_boost_round=100, verbose=True
):
    """
    Perform cross-validation using a LightGBM model and collect feature importances.
    
    Parameters:
        params (dict): LightGBM training parameters.
        X (DataFrame): Feature dataset.
        y (Series): Target variable.
        kf (KFold): KFold cross-validation splitter.
        num_boost_round (int): Number of boosting rounds.
        verbose (bool): If True, prints timing information.

    Returns:
        scores (list): RMSE scores for each fold.
        feature_importance_df (DataFrame): Average (and per-fold) feature importances.
    """
    # Prepare a DataFrame to store feature importances for each fold
    feature_importance_df = pd.DataFrame(
        np.zeros((X.shape[1], kf.get_n_splits())),
        index=X.columns
    )
    
    scores = []

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        if verbose:
            print(f"Starting Fold {fold_idx}...")

        X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]

        # Prepare LightGBM datasets
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

        # Train the model
        fit_model = lgb.train(
            params,
            train_data,
            num_boost_round=num_boost_round,
            valid_sets=[valid_data],
        )

        # Predict
        y_pred = fit_model.predict(X_valid, num_iteration=fit_model.best_iteration)
        rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
        scores.append(rmse)

        # Store feature importances
        fold_importances = fit_model.feature_importance(importance_type='gain')
        feature_importance_df.iloc[:, fold_idx - 1] = fold_importances

        if verbose:
            print(f"Fold {fold_idx} RMSE: {rmse:.4f}")
            print("-" * 50)

    # Compute average feature importance across folds
    feature_importance_df['mean_importance'] = feature_importance_df.mean(axis=1)
    feature_importance_df.sort_values('mean_importance', ascending=False, inplace=True)


    if verbose:
        print("Cross-validation RMSE per fold:", scores)
        print("Average RMSE across folds:", np.mean(scores))

    return scores, feature_importance_df

In [32]:
kf = KFold(n_splits=20, shuffle=True, random_state=42)
lightgbm_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.95,
    'verbose': -1,
    'force_row_wise': True
}

# scores, feature_importances = cross_validate_lightgbm_feature_importances(lightgbm_params, X, y, kf)
# feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
# feature_importances.head(20)  # top 20 most important features

In [54]:
X

,brand,material,size,compartments,laptop_compartment,is_waterproof,style,color,weight_capacity,size_int,weight_capacity_int,weight_capacity_size,weight_capacity_binned,weight_capacity_brand,weight_capacity_poly_2,weight_capacity_poly_3,weight_capacity_exp,weight_capacity_inv,weight_capacity_pca
0,Jansport,Leather,Medium,7.0,1,0,Tote,Black,11.611723,2,11,23.223446,Low,11.611723,134.832107,1565.633046,1.103843e+05,0.086120,6.409977
1,Jansport,Canvas,Small,10.0,1,1,Messenger,Green,27.078537,1,27,27.078537,Very High,27.078537,733.247143,19855.259594,5.755181e+11,0.036930,-9.056836
2,Under Armour,Leather,Small,2.0,1,0,Messenger,Red,16.643760,1,16,16.643760,Medium,66.575040,277.014745,4610.566922,1.691582e+07,0.060083,1.377940
3,Nike,Nylon,Small,8.0,1,0,Messenger,Green,12.937220,1,12,12.937220,Medium,25.874441,167.371669,2165.324158,4.154927e+05,0.077296,5.084480
4,Adidas,Canvas,Medium,1.0,1,1,Messenger,Green,17.749338,2,17,35.498677,Medium,0.000000,315.039016,5591.734125,5.110222e+07,0.056340,0.272362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,Adidas,Leather,Small,9.0,0,0,Tote,Blue,12.730812,1,12,12.730812,Medium,0.000000,162.073567,2063.328072,3.380036e+05,0.078550,5.290888
299996,Jansport,Leather,Large,6.0,0,1,Tote,Blue,26.633182,3,26,79.899547,Very High,26.633182,709.326396,18891.619165,3.686753e+11,0.037547,-8.611482
299997,Puma,Canvas,Large,9.0,1,1,Backpack,Pink,11.898250,3,11,35.694749,Low,35.694749,141.568346,1684.415531,1.470091e+05,0.084046,6.123450
299998,Adidas,Nylon,Small,1.0,0,1,Tote,Pink,6.175738,1,6,6.175738,Low,0.000000,38.139739,235.541029,4.809378e+02,0.161924,11.845962


In [10]:
X.columns

Index(['product_id', 'brand', 'material', 'size', 'num_compartments',
       'laptop_compartment', 'is_waterproof', 'style', 'color',
       'weight_capacity_kg'],
      dtype='object')

In [33]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X, 
    y, kf, verbose=False
)
display(feature_importances[['mean_importance']])
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
weight_capacity_kg,4.717618e+07,4.721959e+07,4.678176e+07,4.736137e+07,4.817562e+07,4.779043e+07,4.783330e+07,4.594700e+07,4.658806e+07,4.652244e+07,...,4.584865e+07,4.819638e+07,4.720334e+07,4.756474e+07,4.550435e+07,4.525354e+07,4.671962e+07,4.520645e+07,4.653646e+07,4.683357e+07
color,1.140844e+07,1.123151e+07,1.136385e+07,1.115432e+07,1.101398e+07,1.120275e+07,1.112198e+07,1.127123e+07,1.146927e+07,1.148296e+07,...,1.126256e+07,1.123969e+07,1.140932e+07,1.163309e+07,1.128278e+07,1.136337e+07,1.134959e+07,1.129527e+07,1.136377e+07,1.131610e+07
num_compartments,9.850175e+06,9.953490e+06,1.048017e+07,1.041116e+07,9.966054e+06,9.856776e+06,9.937975e+06,1.050331e+07,1.005885e+07,1.031951e+07,...,9.777032e+06,1.098234e+07,1.009648e+07,1.003705e+07,9.804528e+06,1.058081e+07,1.078521e+07,1.023005e+07,9.706605e+06,1.015966e+07
brand,9.751406e+06,1.022066e+07,1.038163e+07,1.020107e+07,9.409825e+06,1.007946e+07,1.018233e+07,1.090571e+07,1.019280e+07,1.009509e+07,...,1.020975e+07,1.009005e+07,9.548218e+06,1.023404e+07,9.919946e+06,9.963720e+06,9.688279e+06,1.010095e+07,1.027983e+07,1.007738e+07
material,1.006106e+07,9.759496e+06,9.451678e+06,1.001424e+07,9.734259e+06,9.532279e+06,9.919994e+06,9.546287e+06,9.655160e+06,9.750221e+06,...,9.695774e+06,9.768202e+06,9.669919e+06,1.003611e+07,9.613994e+06,9.413702e+06,9.627305e+06,9.469726e+06,9.694849e+06,9.692751e+06
is_waterproof,5.646812e+06,5.642205e+06,5.065810e+06,5.204073e+06,4.936562e+06,4.538939e+06,5.441004e+06,5.383234e+06,5.183691e+06,4.246944e+06,...,5.418283e+06,4.870393e+06,5.550657e+06,4.927017e+06,5.267349e+06,4.972105e+06,5.453935e+06,4.874958e+06,4.646908e+06,5.093478e+06
size,4.629158e+06,4.873048e+06,4.844941e+06,4.590188e+06,4.904792e+06,5.102831e+06,4.580255e+06,4.994526e+06,4.647928e+06,4.899191e+06,...,4.846777e+06,4.716332e+06,5.059158e+06,4.549812e+06,4.759914e+06,4.594763e+06,4.699014e+06,4.826413e+06,4.790805e+06,4.775271e+06
noise_norm,3.858605e+06,3.600125e+06,3.853762e+06,3.774520e+06,4.063042e+06,3.939191e+06,4.178238e+06,3.828656e+06,3.650533e+06,3.899833e+06,...,3.955847e+06,3.535573e+06,3.967788e+06,3.691078e+06,4.240877e+06,4.265998e+06,3.779018e+06,4.290970e+06,3.903421e+06,3.899414e+06
product_id,3.576922e+06,3.945006e+06,4.036544e+06,4.135792e+06,3.926440e+06,4.065940e+06,3.740384e+06,3.820629e+06,3.878249e+06,4.025878e+06,...,3.799571e+06,3.884763e+06,3.940938e+06,3.903273e+06,3.716683e+06,3.707034e+06,3.992994e+06,3.881061e+06,3.758094e+06,3.867014e+06
noise_uniform,4.099976e+06,3.302216e+06,3.464669e+06,3.441611e+06,3.647948e+06,3.510186e+06,3.483155e+06,3.980496e+06,3.867991e+06,3.970294e+06,...,4.091708e+06,3.842930e+06,3.900921e+06,3.460060e+06,3.725567e+06,3.670027e+06,3.698930e+06,3.737291e+06,3.672984e+06,3.719553e+06


Average RMSE across folds: 38.885562132662876


In [17]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X, 
    y, kf, verbose=False
)
display(feature_importances)
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
weight_capacity_kg,4.970836e+07,5.009177e+07,4.881188e+07,4.993966e+07,5.043774e+07,5.039478e+07,4.929307e+07,4.832035e+07,4.961203e+07,5.004066e+07,...,4.889107e+07,5.112134e+07,5.035884e+07,5.048694e+07,4.852932e+07,4.905535e+07,4.890213e+07,4.848075e+07,4.829892e+07,4.949578e+07
color,1.146010e+07,1.159777e+07,1.185324e+07,1.152232e+07,1.146149e+07,1.199062e+07,1.155820e+07,1.169360e+07,1.158795e+07,1.154271e+07,...,1.179715e+07,1.142582e+07,1.171969e+07,1.185441e+07,1.134933e+07,1.167503e+07,1.176387e+07,1.173074e+07,1.160561e+07,1.163065e+07
num_compartments,1.092418e+07,1.079111e+07,1.077524e+07,1.088446e+07,1.121763e+07,1.084459e+07,1.134071e+07,1.122805e+07,1.101785e+07,1.102964e+07,...,1.067138e+07,1.170532e+07,1.050696e+07,1.107028e+07,1.108721e+07,1.083003e+07,1.167028e+07,1.091614e+07,1.084455e+07,1.099999e+07
brand,1.015740e+07,1.014807e+07,1.073866e+07,1.037227e+07,9.817144e+06,1.016096e+07,1.074167e+07,1.066086e+07,1.027984e+07,1.069148e+07,...,1.085354e+07,1.068638e+07,1.028863e+07,1.064358e+07,1.015250e+07,1.068022e+07,1.046195e+07,1.077928e+07,1.102790e+07,1.047932e+07
material,1.039614e+07,1.026724e+07,1.003439e+07,1.041373e+07,1.030775e+07,9.847701e+06,1.017364e+07,1.012645e+07,1.012117e+07,1.002612e+07,...,1.008641e+07,1.039705e+07,1.049590e+07,1.041907e+07,1.049448e+07,9.845522e+06,1.006205e+07,9.915087e+06,1.001450e+07,1.016586e+07
size,5.115050e+06,5.060329e+06,5.410241e+06,5.279154e+06,4.880583e+06,5.649601e+06,5.258076e+06,5.058771e+06,5.117460e+06,5.254743e+06,...,5.121037e+06,4.818203e+06,5.112423e+06,5.409638e+06,5.215485e+06,5.229924e+06,5.135211e+06,5.033824e+06,5.110233e+06,5.161405e+06
is_waterproof,5.227879e+06,4.637209e+06,5.132789e+06,5.047952e+06,5.693232e+06,4.534978e+06,5.910206e+06,5.505472e+06,4.651608e+06,4.376571e+06,...,5.626157e+06,4.017785e+06,5.388030e+06,4.767879e+06,4.994341e+06,5.005151e+06,5.706819e+06,4.845218e+06,5.408251e+06,5.120108e+06
product_id,5.242075e+06,5.138895e+06,5.173410e+06,4.943708e+06,4.933860e+06,4.790186e+06,5.108704e+06,5.543346e+06,5.442714e+06,5.130544e+06,...,5.224038e+06,4.831928e+06,5.283092e+06,5.153767e+06,4.882136e+06,4.958511e+06,4.908895e+06,5.165048e+06,5.038287e+06,5.111515e+06
laptop_compartment,1.486326e+06,1.479301e+06,1.761428e+06,1.736026e+06,1.442151e+06,1.653900e+06,1.227560e+06,1.901829e+06,1.453008e+06,1.672447e+06,...,1.873241e+06,1.595976e+06,1.352292e+06,1.438032e+06,1.633936e+06,1.546898e+06,1.253184e+06,2.347272e+06,1.733061e+06,1.613927e+06
style,1.566064e+06,1.783020e+06,1.768112e+06,1.337933e+06,1.416998e+06,1.450940e+06,1.546021e+06,1.238834e+06,1.680685e+06,1.421421e+06,...,1.515651e+06,1.531387e+06,1.520441e+06,1.378670e+06,1.510594e+06,1.764205e+06,1.693717e+06,1.606759e+06,1.408614e+06,1.526128e+06


Average RMSE across folds: 38.88401308882421


In [36]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['noise_uniform']], 
    y, kf, verbose=False
)
display(feature_importances[['mean_importance']])
print(f"Average RMSE across folds: {np.mean(scores):.4f}")

,mean_importance
noise_uniform,1.711164e+06


Average RMSE across folds: 38.93977597829933


In [37]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['product_id']], 
    y, kf, verbose=False
)
display(feature_importances[['mean_importance']])
print(f"Average RMSE across folds: {np.mean(scores):.4f}")

,mean_importance
product_id,2.053748e+06


Average RMSE across folds: 38.939532938793356


In [39]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg']], 
    y, kf, verbose=False
)
display(feature_importances[['mean_importance']])
print(f"Average RMSE across folds: {np.mean(scores):.4f}")

,mean_importance
weight_capacity_kg,4.299855e+07


Average RMSE across folds: 38.9133


In [40]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color']], 
    y, kf, verbose=False
)
display(feature_importances[['mean_importance']])
print(f"Average RMSE across folds: {np.mean(scores):.4f}")

,mean_importance
weight_capacity_kg,4.470421e+07
color,1.146484e+07


Average RMSE across folds: 38.9087


In [41]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color', 'num_compartments']], 
    y, kf, verbose=False
)
display(feature_importances[['mean_importance']])
print(f"Average RMSE across folds: {np.mean(scores):.4f}")

,mean_importance
weight_capacity_kg,4.614248e+07
color,1.193505e+07
num_compartments,9.031126e+06


Average RMSE across folds: 38.9047


In [42]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color', 'num_compartments', 'brand']], 
    y, kf, verbose=False
)
feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
display(feature_importances[['mean_importance']])
print(f"Average RMSE across folds: {np.mean(scores):.4f}")

,mean_importance
weight_capacity_kg,4.587654e+07
color,1.200961e+07
brand,9.593835e+06
num_compartments,8.953673e+06


Average RMSE across folds: 38.9008


In [43]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color', 'num_compartments', 'brand', 'material']], 
    y, kf, verbose=False
)
display(feature_importances[['mean_importance']])
print(f"Average RMSE across folds: {np.mean(scores):.4f}")

,mean_importance
weight_capacity_kg,4.549920e+07
color,1.206680e+07
brand,9.855395e+06
material,9.655583e+06
num_compartments,8.719744e+06


Average RMSE across folds: 38.8963


In [44]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color', 'num_compartments', 'brand', 'material', 'is_waterproof']], 
    y, kf, verbose=False
)
display(feature_importances[['mean_importance']])
print(f"Average RMSE across folds: {np.mean(scores):.4f}")

,mean_importance
weight_capacity_kg,4.763698e+07
color,1.227628e+07
material,9.968583e+06
brand,9.720940e+06
num_compartments,9.358182e+06
is_waterproof,5.934946e+06


Average RMSE across folds: 38.8909
